=====DATSCIW261 ASSIGNMENT #5=====

MIDS UC Berkeley, Machine Learning at Scale

DATSCIW261 ASSIGNMENT #5

Version 2016-02-12 (FINAL)


#### Hetal Chandaria (hetalchandaria@berkeley.edu) 

W261 - 2 , ASSIGNMENT #5

Submission Date : 

Group : 4

#### HW 5.0
What is a data warehouse? What is a Star schema? When is it used?

#### HW 5.1
In the database world What is 3NF? Does machine learning use data in 3NF? If so why? 
In what form does ML consume data?
Why would one use log files that are denormalized?

#### HW 5.2
Using MRJob, implement a hashside join (memory-backed map-side) for left, 
right and inner joins. Run your code on the  data used in HW 4.4: (Recall HW 4.4: Find the most frequent visitor of each page using mrjob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.)
:

Justify which table you chose as the Left table in this hashside join.

Please report the number of rows resulting from:

(1) Left joining Table Left with Table Right

(2) Right joining Table Left with Table Right

(3) Inner joining Table Left with Table Right

<span style="color: blue"> Answer: </span>

right table = log table of the format page id, customer id 
left table = url table of the format page id, url

The left table is the Page Id's & the corresponding URL,because in memory backed map-side joins the left table is the smaller table of the two tables we want to join.


In [111]:
%%writefile leftjoin.py

from mrjob.job import MRJob
from mrjob.step import MRJobStep
import csv, re
from mrjob.protocol import RawProtocol, ReprProtocol

class LeftJoin(MRJob):
    left ={}
    left_seen = {}
    def steps(self):
        return [
            MRJobStep(mapper_init=self.mapper_init, mapper=self.mapper,mapper_final = self.mapper_final)
               ]
    
    def mapper_init(self):
        with open('most_visitors.out') as f:
            self.left = {k: v for line in f for (k, v) in (line.replace('"', '').strip().split('\t'),)}
        self.left_seen = dict.fromkeys(self.left, 0)
    
    def mapper(self, line_no, line):
        cell = line.strip().split(',')
        cust_id = self.left.get(cell[0],'NA')
        if cust_id != 'NA':
            self.left_seen[cell[0]] = 1
            yield cell[0], (cell[1],cust_id)
    
    def mapper_final(self):
        for pageid, seen in self.left_seen.iteritems():
            if seen == 1:
                continue
            yield pageid, (self.left.get(pageid),'NULL')
                
                   
if __name__ == '__main__':
    LeftJoin.run()

Overwriting leftjoin.py


In [112]:
!python ./leftjoin.py processed_urls.data -q >left.out

print "Left Join"

print "Number of rows for Left join:" 
!wc -l left.out |awk '{print $1}'
print "\nSample data"
!head left.out

Left Join
Number of rows for Left join:
285

Sample data
"1215"	["/developer", "12577"]
"1279"	["/msgolf", "31062"]
"1239"	["/msconsult", "15361"]
"1282"	["/home", "39877"]
"1251"	["/referencesupport", "18941"]
"1121"	["/magazine", "10241"]
"1083"	["/msaccesssupport", "10127"]
"1145"	["/vfoxprosupport", "10418"]
"1276"	["/vtestsupport", "29654"]
"1200"	["/benelux", "11674"]


In [109]:
%%writefile innerjoin.py

from mrjob.job import MRJob
from mrjob.step import MRJobStep
import csv, re
from mrjob.protocol import RawProtocol, ReprProtocol

class InnerJoin(MRJob):
    def steps(self):
        return [
            MRJobStep(mapper_init=self.mapper_init, mapper=self.mapper)
               ]
    
    def mapper_init(self):
        with open('most_visitors.out') as f:
            self.left = {k: v for line in f for (k, v) in (line.replace('"', '').strip().split('\t'),)}
        
    
    def mapper(self, line_no, line):
        cell = line.strip().split(',')
        url = self.left.get(cell[0],'NA')
        if url != 'NA':
            yield cell[0], (cell[1],self.left.get(cell[0],'NA'))
                   
if __name__ == '__main__':
    InnerJoin.run()

Overwriting innerjoin.py


In [110]:
!python ./innerjoin.py processed_urls.data -q >inner.out

print "Inner Join"

print "Number of rows for inner join:" 
!wc -l inner.out |awk '{print $1}'
print "\nSample data"
!head inner.out

Inner Join
Number of rows for inner join:
285

Sample data
"1215"	["/developer", "12577"]
"1279"	["/msgolf", "31062"]
"1239"	["/msconsult", "15361"]
"1282"	["/home", "39877"]
"1251"	["/referencesupport", "18941"]
"1121"	["/magazine", "10241"]
"1083"	["/msaccesssupport", "10127"]
"1145"	["/vfoxprosupport", "10418"]
"1276"	["/vtestsupport", "29654"]
"1200"	["/benelux", "11674"]


In [107]:
%%writefile rightjoin.py

from mrjob.job import MRJob
from mrjob.step import MRJobStep
import csv, re
from mrjob.protocol import RawProtocol, ReprProtocol

class RightJoin(MRJob):
    def steps(self):
        return [
            MRJobStep(mapper_init=self.mapper_init, mapper=self.mapper)
               ]
    
    def mapper_init(self):
        with open('most_visitors.out') as f:
            self.left = {k: v for line in f for (k, v) in (line.replace('"', '').strip().split('\t'),)}
    
    def mapper(self, line_no, line):
        cell = line.strip().split(',')
        yield cell[0], (cell[1],self.left.get(cell[0],'NULL'))
                   
if __name__ == '__main__':
    RightJoin.run()

Overwriting rightjoin.py


In [108]:
!python ./rightjoin.py processed_urls.data -q >right.out

print "Right Join"

print "Number of rows for Right join:" 
!wc -l right.out |awk '{print $1}'
print "\nSample data"
!head right.out

Right Join
Number of rows for Right join:
294

Sample data
"1287"	["/autoroute", "NULL"]
"1288"	["/library", "NULL"]
"1289"	["/masterchef", "NULL"]
"1297"	["/centroam", "NULL"]
"1215"	["/developer", "12577"]
"1279"	["/msgolf", "31062"]
"1239"	["/msconsult", "15361"]
"1282"	["/home", "39877"]
"1251"	["/referencesupport", "18941"]
"1121"	["/magazine", "10241"]
